# 필요한 라이브러리 로드

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import pandas as pd
from time import sleep
from tqdm import tqdm

# 초기 화면 로드

In [2]:
keyword = '아이즈원'
url = 'https://www.youtube.com/results?search_query={}'.format(keyword)
s = Service('./chromedriver.exe')
driver = webdriver.Chrome(service=s)
driver.get(url)
action = ActionChains(driver)

# 동영상만 보기 설정

In [3]:
filter_element = WebDriverWait(driver, 10).until(
 EC.presence_of_all_elements_located((By.XPATH, "//*[@id='text']"))
)

filter_element[2].click()

In [4]:
filter_sub_element = WebDriverWait(driver, 10).until(
 EC.presence_of_all_elements_located((By.XPATH, "//*[contains(text(),'동영상')]"))
)

filter_sub_element[1].click()

# 스크롤 100회 내리기

In [5]:
for i in tqdm(range(40)):
    start = i * 10000
    end = (i+1) * 10000
    driver.execute_script(f"window.scrollTo({start},{end})")
    sleep(2)

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [01:22<00:00,  2.05s/it]


# 제목, 업로더 크롤링

## 제목, 업로더 태그 가져오기

In [6]:
izone_titles = driver.find_elements(By.ID, "video-title")
izone_uploader= driver.find_elements(By.XPATH, "//a[@class='yt-simple-endpoint style-scope yt-formatted-string']")

## 제목, 업로더 텍스트 저장

In [7]:
titles = [x.text for x in izone_titles]
uploaders = [i.text for i in izone_uploader if i.text != '']

# 조회수, 날짜 크롤링

In [8]:
error_row = []

In [9]:
izone_list = []
views = []
update_dates = []
    
for i,title in tqdm(enumerate(izone_titles)):
    try:
        
        title_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable(title)
        )
        
        title_element.send_keys(Keys.CONTROL +"\n")
        driver.switch_to.window(driver.window_handles[1])
        
        view_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//span[@class='view-count style-scope ytd-video-view-count-renderer']"))
        )
        
        update_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//div[@id='info-strings']"))
        )
        
        views.append(view_element.text)
        update_dates.append(update_element.text)

        driver.close()

        # 메인 탭으로 이동
        driver.switch_to.window(driver.window_handles[0])
        
    except:
        error_row.append(i)
        views.append('오류')
        update_dates.append('오류')
        driver.close()

        # 메인 탭으로 이동
        driver.switch_to.window(driver.window_handles[0])

544it [16:58,  1.87s/it]


In [10]:
error_row

[294, 536, 539, 543]

In [11]:
for title,uploader,view,update_date in zip(titles,uploaders,views,update_dates):
    izone_list.append([title,uploader,view,update_date])

In [12]:
izone_list

[['엠넷 재팬 MC 本田仁美 혼다히토미 아이즈원 언급 한글자막',
  '곽군테레비 郭君テレビ',
  '조회수 11,330회',
  '2021. 12. 28.'],
 ['아이즈원', '강민혁 X 차누', '조회수 3,727,913회', '2021. 3. 25.'],
 ['🎵 아이즈원 (IZONE) 노래모음 추천곡 (라비앙로즈, FIESTA, 비올레타, 환상동화, Airplane, 하늘 위로, 해바라기 ..)',
  '박승오리 Box',
  '조회수 158,297회',
  '2020. 11. 5.'],
 ['아이즈원(IZ*ONE) - Panorama(파노라마) 교차편집(stage mix)',
  '강민혁 X 차누',
  '조회수 5,157,126회',
  '2021. 3. 24.'],
 ["IZ*ONE (아이즈원) 'Panorama' MV",
  'Stone Music Entertainment',
  '조회수 69,191,968회',
  '2020. 12. 7.'],
 ["[4회] 아홉 개의 보석! '크라운' ♬FIESTA_IZ*ONE(아이즈원) @CONNECT MISSION #GirlsPlanet999 | Mnet 210827 방송",
  'Mnet K-POP',
  '조회수 9,694,389회',
  '2021. 8. 27.'],
 ['[ENG/JPN] 🧐마피아게임 하다가 단체로 드러누운(?) 아이즈원 | IZ*ONEㅣPanoramaㅣ마피아 댄스 | 딩고뮤직ㅣDingo MusicㅣMAFIA DANCE',
  '딩고 뮤직 / dingo music',
  '조회수 5,807,974회',
  '2020. 12. 13.'],
 ['하이브 걸그룹 새로운 멤버 등장!!? 특급 소식 떴다 !!아이브는 너무 잘나가는데..',
  '아이돌로그',
  '조회수 28,276회',
  '2021. 12. 27.'],
 ["아이즈원 (IZ*ONE) '비올레타 (Violeta)' 교차편집 (Stage Mix)",
  '호구마 HOGUMA',
  '조회수 8,302,657회',
  '

In [13]:
izone_df = pd.DataFrame(data=izone_list,columns=['제목','업로더','조회수','게시일'])

In [14]:
izone_df = izone_df.drop_duplicates(keep='last')

In [15]:
izone_df=  izone_df.reset_index(drop=True)

In [16]:
izone_df

,제목,업로더,조회수,게시일
0,엠넷 재팬 MC 本田仁美 혼다히토미 아이즈원 언급 한글자막,곽군테레비 郭君テレビ,"조회수 11,330회",2021. 12. 28.
1,아이즈원,강민혁 X 차누,"조회수 3,727,913회",2021. 3. 25.
2,"🎵 아이즈원 (IZONE) 노래모음 추천곡 (라비앙로즈, FIESTA, 비올레타, ...",박승오리 Box,"조회수 158,297회",2020. 11. 5.
3,아이즈원(IZ*ONE) - Panorama(파노라마) 교차편집(stage mix),강민혁 X 차누,"조회수 5,157,126회",2021. 3. 24.
4,IZ*ONE (아이즈원) 'Panorama' MV,Stone Music Entertainment,"조회수 69,191,968회",2020. 12. 7.
...,...,...,...,...
535,Cruella,YouTube 영화,오류,오류
536,2021년 베스트 걸그룹 1위!! 아이즈원 삼성동 광고,곽군테레비 郭君テレビ,"조회수 1,916회",2021. 12. 29.
537,[아이즈원]드디어 뭉쳤다 3사 MC즈!,쌈조TV3,"조회수 22,516회",2021. 12. 31.
538,아이즈원 굿즈 판매! 완전 싸게 처분!,아쪼,조회수 26회,2021. 12. 30.


In [17]:
izone_df.to_csv('./data/izone_youtube.csv',sep=',',na_rep='NaN',encoding='utf-8-sig')